In [1]:
p = property(fget=lambda self: print('Getting property...'))

In [2]:
p

In [3]:
property.__dict__

mappingproxy({'__getattribute__': <slot wrapper '__getattribute__' of 'property' objects>,
              '__get__': <slot wrapper '__get__' of 'property' objects>,
              '__set__': <slot wrapper '__set__' of 'property' objects>,
              '__delete__': <slot wrapper '__delete__' of 'property' objects>,
              '__init__': <slot wrapper '__init__' of 'property' objects>,
              '__new__': <function property.__new__(*args, **kwargs)>,
              'getter': <method 'getter' of 'property' objects>,
              'setter': <method 'setter' of 'property' objects>,
              'deleter': <method 'deleter' of 'property' objects>,
              'fget': <member 'fget' of 'property' objects>,
              'fset': <member 'fset' of 'property' objects>,
              'fdel': <member 'fdel' of 'property' objects>,
              '__doc__': <member '__doc__' of 'property' objects>,
              '__isabstractmethod__': <attribute '__isabstractmethod__' of 'property' objec

## Recap of Decorators in Python

In [15]:
def my_decorator(func):
    print('Decorating function')
    # closure --
    def inner(*args, **kwargs):      # Wrapper function --
        print('Running decorated function..')
        # func is a free variable, which is in this case is original function
        result = func(*args, **kwargs)
        return result
    return inner

def my_func(a, b):
    print('running original function--')
    return a+b

In [16]:
decorated_func = my_decorator(my_func)
# It is equivalent to wrapper function --
print(decorated_func)

# run it --
decorated_func(5,6)

Decorating function
<function my_decorator.<locals>.inner at 0x059489C0>
Running decorated function..
running original function--


11

**Decorated way --**

In [17]:
def my_decorator(func):
    print('Decorating function')
    # closure --
    def inner(*args, **kwargs):      # Wrapper function --
        print('Running decorated function..')
        # func is a free variable, which is in this case is original function
        result = func(*args, **kwargs)
        return result
    return inner

@my_decorator
def my_func(a, b):
    print('running original function--')
    return a+b

Decorating function


In [18]:
my_func(5,6)

Running decorated function..
running original function--


11

### Why Property can be used as a decorator ?

Let's look at it syntax -<br>
```p = property(fget=lambda self: print('Getting property...'))```<br>

we are taking some function and wrapping it around property() and returning an object out of it.

And this is the same we do when we use decorators.

In [19]:
class Person:
    def __init__(self, name):
        self._name = name
        
    def name(self):
        print('=== Getter called ===')
        return self._name
    
    name = property(fget=name)

In [20]:
p = Person('Sarthak')
p.name

=== Getter called ===


'Sarthak'

**Decorated way--**

In [36]:
class Person:
    def __init__(self, name):
        self._name = name
    
    # Getter method--
    @property
    def name(self):
        print('=== Getter called ===')
        return self._name

In [37]:
p = Person('Sarthak')
p.name

=== Getter called ===


'Sarthak'

<h3>Section: Concept</h3> 
**Lets get into some details(basic)**
<br>It will help you to understand How to use setter method as a decorator.

In [38]:
def get_prop(self):
    return 'Getter Called --'

def set_prop(self, value):
    return 'Setter called --'

def del_prop(self):
    return 'Deleter called --'

p = property(get_prop)

In [39]:
# fset is None as we didn't assign any setter method yet
p.fget, p.fset

(<function __main__.get_prop(self)>, None)

**Lets assign setter method to new variable**

In [40]:
p1 = p.setter(set_prop)

In [41]:
p.fset, p1.fget, p1.fset

(None,
 <function __main__.get_prop(self)>,
 <function __main__.set_prop(self, value)>)

See, by assigning it to different variable. We get, two different property objects(```p``` and ```p1```), and becuase of that ```p``` object only has getter but not setter, which might results into anamolies when we are dealing with complex classes and objects.

**Lets assign setter method to its own variable**

In [42]:
p = p.setter(set_prop)
p.fset, p.fget

# now p object has both getter and setter.

(<function __main__.set_prop(self, value)>, <function __main__.get_prop(self)>)

<h3> Section Concept ends </h3>

Now observe ```p.setter``` is a function which wraps ```set_prop``` and it is returning a object

**So it can be used as a decorator**
<br>Lets see--

In [47]:
class Person:
    def __init__(self, name):
        self._name = name
    
    @property
    def name(self):
        print("Getter called ---")
        return self._name
    
    @name.setter
    def name(self, value):
        print("Setter called ---")
        self._name = value

In [48]:
p = Person('Alex')
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              'name': <property at 0x6fb6750>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

Observe, it is having only one property object i.e ```name```

In [49]:
p.name

Getter called ---


'Alex'

In [50]:
p.name = 'Sarthak'

Setter called ---


<h3> Cool Everything is working great! </h3>

But now let me show you why function(inside setter decorator) must be of a same name as a getter. 

In [51]:
class Person:
    def __init__(self, name):
        self._name = name
    
    @property
    def name(self):
        print("Getter called ---")
        return self._name
    
    # I am using different function name --
    @name.setter
    def set_name(self, value):
        print("Setter called ---")
        self._name = value

In [52]:
p = Person('Alex')
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              'name': <property at 0x700bb10>,
              'set_name': <property at 0x700bdb0>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

**Note:** Now, we are having two different property objects, ```name``` and ```set_name```

In [53]:
p.name

Getter called ---


'Alex'

In [54]:
# Now this time we are getting error, and that's only because we didn't
# assing setter back to name object
# Remember: In Section Concept: for p object we didn't have setter, but p1 does.

p.name = 'Sarthak'

AttributeError: can't set attribute

In [55]:
# we can use setter method with another object i.e set_name
p.set_name = 'Sarthak'

Setter called ---


So, now you can think yourself that this can be really confusing sometimes.

So it is better to use same function names